In [43]:
import mediapipe as mp
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np


mp_draw = mp.solutions.drawing_utils
mp_draw_styles = mp.solutions.drawing_styles

def draw_landmarks_on_image(rgb_image, face_landmarks):
    annotated_image = np.copy(rgb_image)

    # Draw the face landmarks.
    face_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    face_landmarks_proto.landmark.extend([
        landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in face_landmarks
    ])
    # # 人脸网格
    # solutions.drawing_utils.draw_landmarks(
    #     image=annotated_image,
    #     landmark_list=face_landmarks_proto,
    #     connections=mp.solutions.face_mesh.FACEMESH_TESSELATION,
    #     landmark_drawing_spec=None,
    #     connection_drawing_spec=mp.solutions.drawing_styles
    #     .get_default_face_mesh_tesselation_style())
    # # 脸廓、眼睫毛、眼眶、嘴唇
    # solutions.drawing_utils.draw_landmarks(
    #     image=annotated_image,
    #     landmark_list=face_landmarks_proto,
    #     connections=mp.solutions.face_mesh.FACEMESH_CONTOURS,
    #     landmark_drawing_spec=None,
    #     connection_drawing_spec=mp.solutions.drawing_styles
    #     .get_default_face_mesh_contours_style())
    # # 瞳孔
    # solutions.drawing_utils.draw_landmarks(
    #     image=annotated_image,
    #     landmark_list=face_landmarks_proto,
    #     connections=mp.solutions.face_mesh.FACEMESH_IRISES,
    #     landmark_drawing_spec=None,
    #     connection_drawing_spec=mp.solutions.drawing_styles
    #     .get_default_face_mesh_iris_connections_style())
    # 关键点
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        landmark_drawing_spec=mp_draw.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=1))

    return annotated_image

In [44]:
left_eye = [33, 246, 161, 160, 159, 158, 157, 173, 133, 155, 154, 153, 145, 144, 163, 7, 33]
right_eye = [263, 466, 388, 387, 386, 385, 384, 398, 362, 382, 381, 380, 374, 373, 390, 249]
mouth = [61, 185, 40, 39, 37, 0, 267, 270, 409, 291, 375, 321, 405, 314, 17, 84, 181, 91, 146]
eye_center = [468, 473]
mid = [9, 8, 168, 6, 197, 195, 5, 4, 1, 19, 2, 164, 0, 11, 12, 13, 14, 15, 16, 17, 18, 200]
contours = [234, 454]

def process_face_landmarks(face_landmarks):
    processed_face_landmarks = []
    for idx in left_eye:
        processed_face_landmarks.append(face_landmarks[idx])
    for idx in right_eye:
        processed_face_landmarks.append(face_landmarks[idx])
    for idx in mouth:
        processed_face_landmarks.append(face_landmarks[idx])
    for idx in eye_center:
        processed_face_landmarks.append(face_landmarks[idx])
    for idx in mid:
        processed_face_landmarks.append(face_landmarks[idx])
    for idx in contours:
        processed_face_landmarks.append(face_landmarks[idx])
    return processed_face_landmarks

In [45]:
import mediapipe as mp
import cv2
import math
import time

BaseOptions = mp.tasks.BaseOptions
FaceLandmarker = mp.tasks.vision.FaceLandmarker
FaceLandmarkerOptions = mp.tasks.vision.FaceLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode


# Create a face landmarker instance with the video mode:
model_path = 'face_landmarker.task'
options = FaceLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.VIDEO)


with FaceLandmarker.create_from_options(options) as landmarker:
    # 打开视频文件
    video_path = '../data/processed/L1/6-ce.mp4'  # 替换为你的视频文件路径
    cap = cv2.VideoCapture(video_path)

    # 获取视频的帧率和总帧数
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    h, w = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)), int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))

    print(f"Video FPS: {fps}")
    print(f"Total Frames: {total_frames}")
    print(f"Video Resolution: {w}x{h}")

    n = 1
    # 读取并显示每一帧
    while cap.isOpened():
        ret, frame = cap.read()
        n += 1

        # 检查是否成功读取帧
        if not ret:
            break

        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
        face_landmarker_result = landmarker.detect_for_video(mp_image, math.floor(n * 1000 / fps))
        face_landmarks = face_landmarker_result.face_landmarks[0]

        # for landmark in face_landmarks:
        #     cx = int(landmark.x * w)
        #     cy = int(landmark.y * h)
        #     print(landmark.x, landmark.y, landmark.z)
        #     print(cx, cy)

        processed_face_landmarks = process_face_landmarks(face_landmarks)
        annotated_image = draw_landmarks_on_image(frame, processed_face_landmarks)
        cv2.imshow('Video Frame', annotated_image)

        # break
        # 按 'q' 键退出循环
        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

    # 释放资源
    cap.release()
    cv2.destroyAllWindows()

Video FPS: 60.00454249504111
Total Frames: 258
Video Resolution: 1920x1080


In [46]:
help(landmarker)

Help on FaceLandmarker in module mediapipe.tasks.python.vision.face_landmarker object:

class FaceLandmarker(mediapipe.tasks.python.vision.core.base_vision_task_api.BaseVisionTaskApi)
 |  FaceLandmarker(graph_config: mediapipe.framework.calculator_pb2.CalculatorGraphConfig, running_mode: mediapipe.tasks.python.vision.core.vision_task_running_mode.VisionTaskRunningMode, packet_callback: Optional[Callable[[Mapping[str, mediapipe.python._framework_bindings.packet.Packet]], NoneType]] = None) -> None
 |  
 |  Class that performs face landmarks detection on images.
 |  
 |  Method resolution order:
 |      FaceLandmarker
 |      mediapipe.tasks.python.vision.core.base_vision_task_api.BaseVisionTaskApi
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  detect(self, image: mediapipe.python._framework_bindings.image.Image, image_processing_options: Optional[mediapipe.tasks.python.vision.core.image_processing_options.ImageProcessingOptions] = None) -> mediapipe.tasks.python.vision.